![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')
qbr_df = pd.read_csv('data/qbr_season_level.csv') 

stats_df contains player stats by game ill need to get some season totals, career totals, as well as per game stats for passing receiving and rushing. ill use interceptions, special teams tds, and fumbles, sacks to help rank defense. the stats ill use arent missing many entries so after cleaning ill save what i need into a new dataframe. also need to have a cut off for age. if a player is XX y/o they wont be relevant today. ill try to expand on the EPA missing data as that is Expected Points Added which should be extremely beneficial to analysis i should also seperate playoffs and regular season.  

In [3]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,0.292378,0,0.0,0.052632,NaN,NaN,0,12.7,13.7
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,1,0.377009,0,0.0,0.117647,NaN,NaN,0,5.1,8.1
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,-0.699578,0,NaN,0.023810,NaN,NaN,0,0.2,0.2
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,-0.228454,0,0.0,0.050000,NaN,NaN,0,3.5,5.5
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,3.9,3.9


In [14]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129739 entries, 0 to 129738
Data columns (total 69 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   player_id                     129739 non-null  object 
 1   player_name                   61504 non-null   object 
 2   player_display_name           129736 non-null  object 
 3   position                      129667 non-null  object 
 4   position_group                129667 non-null  object 
 5   headshot_url                  69803 non-null   object 
 6   recent_team                   129739 non-null  object 
 7   season                        129739 non-null  int64  
 8   week                          129739 non-null  int64  
 9   season_type                   129739 non-null  object 
 10  opponent_team                 129739 non-null  object 
 11  completions                   129739 non-null  int64  
 12  attempts                      129739 non-nul

In [5]:
stats_df['season'].value_counts()

2021    5698
2023    5653
2022    5631
2020    5447
2012    5354
2014    5350
2017    5319
2015    5318
2011    5301
2018    5281
2016    5274
2019    5261
2009    5242
2013    5231
2010    5201
2003    5142
2004    5129
2002    5078
1999    5031
2001    4895
2000    4874
2007    4841
2008    4814
2006    4712
2005    4662
Name: season, dtype: int64

### Feature Engineering

In [13]:
# Create efficiency metrics
stats_df['completion_percentage'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_attempt'] = stats_df['passing_yards'] / (stats_df['attempts'] + 1)
stats_df['yards_per_carry'] = stats_df['rushing_yards'] / (stats_df['carries'] + 1)
stats_df['yards_per_reception'] = stats_df['receiving_yards'] / (stats_df['receptions'] + 1)

# Create touchdown ratios
stats_df['pass_tds_per_attempt'] = stats_df['passing_tds'] / (stats_df['attempts'] + 1)
stats_df['tds_per_carry'] = stats_df['rushing_tds'] / (stats_df['carries'] + 1)
stats_df['tds_per_reception'] = stats_df['receiving_tds'] / (stats_df['receptions'] + 1)

# Calculate season stats for each player over the season
stats_df['season_passing_yards'] = stats_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
stats_df['season_rushing_yards'] = stats_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
stats_df['season_receiving_yards'] = stats_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
stats_df['season_passing_tds'] = stats_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
stats_df['season_rushing_tds'] = stats_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
stats_df['season_receiving_tds'] = stats_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
stats_df['season_completion_percentage'] = stats_df.groupby(['player_id', 'season'])['completion_percentage'].transform('sum')

# Interaction terms
stats_df['completions_per_attempt'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_touch'] = (stats_df['rushing_yards'] + stats_df['receiving_yards']) / (stats_df['carries'] + stats_df['receptions'] + 1)

#SEPERATE PLAYOFFS

#CUTOFF SEASONS

#CAREER STATS

#USE OPPOSING TEAM TO RECORD D/ST STATS

In [15]:
# checking out well known players with new features
print(stats_df[stats_df['player_display_name'] == 'Tom Brady'])

print(stats_df[stats_df['player_display_name'] == 'Keyshawn Johnson'])

print(stats_df[stats_df['player_display_name'] == 'Curtis Martin'])


         player_id player_name player_display_name position position_group  \
9616    00-0019596     T.Brady           Tom Brady       QB             QB   
13908   00-0019596     T.Brady           Tom Brady       QB             QB   
13909   00-0019596     T.Brady           Tom Brady       QB             QB   
13910   00-0019596     T.Brady           Tom Brady       QB             QB   
13911   00-0019596     T.Brady           Tom Brady       QB             QB   
...            ...         ...                 ...      ...            ...   
118468  00-0019596     T.Brady           Tom Brady       QB             QB   
118469  00-0019596     T.Brady           Tom Brady       QB             QB   
118470  00-0019596     T.Brady           Tom Brady       QB             QB   
118471  00-0019596     T.Brady           Tom Brady       QB             QB   
118472  00-0019596     T.Brady           Tom Brady       QB             QB   

                                             headshot_url recen

## QBR

not sure if this will even be necessary. might be good to have QBR by season to go along with other QB stats

In [8]:
qbr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         1363 non-null   int64  
 1   season_type    1363 non-null   object 
 2   game_week      1363 non-null   object 
 3   team_abb       1363 non-null   object 
 4   player_id      1363 non-null   int64  
 5   name_short     1363 non-null   object 
 6   rank           775 non-null    float64
 7   qbr_total      1363 non-null   float64
 8   pts_added      1363 non-null   float64
 9   qb_plays       1363 non-null   int64  
 10  epa_total      1363 non-null   float64
 11  pass           1363 non-null   float64
 12  run            1363 non-null   float64
 13  exp_sack       1363 non-null   int64  
 14  penalty        1363 non-null   float64
 15  qbr_raw        1363 non-null   float64
 16  sack           1363 non-null   float64
 17  name_first     1363 non-null   object 
 18  name_las

In [9]:
qbr_df['season'].value_counts()

2021    91
2020    84
2023    82
2019    81
2010    80
2022    79
2008    76
2018    74
2016    74
2009    74
2011    73
2013    73
2006    71
2012    71
2015    71
2017    70
2007    70
2014    69
Name: season, dtype: int64

In [ ]:
# GRAB RELEVANT QBR AND CONCATENATE